In [1]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection._validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

1.Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
features = pd.read_csv('features/features.csv', index_col='match_id')
features.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

In [3]:
features_test = pd.read_csv('features_test/features_test.csv', index_col='match_id')
#features_test.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

2.Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
features[features[features.columns].notnull()].count()[lambda x: x < features.count().max()]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

first blood события, говорят нам о том, что первая кровь была сделана в первые пять минуть, а если нет пропуск. Думаю, это связанно со стилем игры проффесионалов, первые 5 минут идет прокачка. А низкое событие связано с тем, что чаще ферст блад происходил на мид лайне, а значит помощники отуствовали.

radiant_flying_courier_time небольшие значения, это связано с тактикой команд, можно предположить, что обычный курьер обходиться дешевле, а т.к. закупка чаще всего связана с таймингом комманды, то необходимости в нем мало.

3.Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. 

Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
features = features.fillna(0)
features_test = features_test.fillna(0)

4.Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
'radiant_win' in features.columns.unique()

False

5.Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [8]:
X_train , Y_train = features.as_matrix(), pd.read_csv('features/features.csv', index_col='match_id')['radiant_win'].as_matrix()
X_test = features_test.as_matrix()
kfold = KFold(n_splits = 5, shuffle=True)

In [13]:
start_time = datetime.datetime.now()
scores=[]
for x in range(1, 51):
    clf = GradientBoostingClassifier(n_estimators=x, random_state=1)
    scores.append(np.array(cross_val_score(clf, X_train, Y_train,scoring='roc_auc', cv=kfold)))
scores=np.array(scores)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:58:40.064712


In [17]:
[np.mean(sc) for sc in scores]

[0.59646101866790802,
 0.61202657373803115,
 0.61883657865657338,
 0.62577175547222796,
 0.63383038110825851,
 0.64490385822372009,
 0.65016098418464274,
 0.65688290044838837,
 0.66116055759621339,
 0.66459800935979951,
 0.66809019000413428,
 0.67065208174746405,
 0.67232130564715908,
 0.67481918224762705,
 0.67513266711541531,
 0.67698857658634859,
 0.67807341498333462,
 0.67974807647378088,
 0.68150848374544526,
 0.68194874970016417,
 0.68271525208243278,
 0.68462154622199378,
 0.68454925251429621,
 0.68614163520246985,
 0.68594831566907466,
 0.68652828540460864,
 0.68643588794778443,
 0.6882198544014303,
 0.68884728727042677,
 0.68988536008597789,
 0.69048125887103995,
 0.69126547519542114,
 0.69043024536260822,
 0.69122651813460367,
 0.69144940104244612,
 0.6922597828332947,
 0.69294670186401219,
 0.69372548630545927,
 0.6935546096969547,
 0.69412698709969689,
 0.69467108299346625,
 0.69467281322475904,
 0.69489906896622888,
 0.69597909908121447,
 0.69554202929923314,
 0.6962665649

In [18]:
np.argmax([np.mean(sc) for sc in scores])+1

50

Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

В целом, можно заметить, что алгоритм улучшается, с каждым разом метрика становится лучше. Но думаю, что это неверно, можно было бы остановиться на 20-30.

Думаю, что я бы брал по 5 оценок, и нахоидл их отклонения. Вычислив минимальный порог эмперически, где качество практически не меняется, отберу нужное количество.

In [28]:
clf.fit(X_train, Y_train)
win = clf.predict_proba(X_test)

In [33]:
pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win']).to_csv('result.scv', index=False)

Score 0.70125